In [1]:
import os

In [2]:
%pwd

'd:\\MLproject\\First_project\\Project\\machine-learning-mlflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\MLproject\\First_project\\Project\\machine-learning-mlflow'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/nishan129/machine-learning-mlflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "nishan129"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "8407753a7583de66d5ed9ef46f18c54448b6824b"

In [16]:
from dataclasses import dataclass
from pathlib import  Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir : Path
    test_data_path : Path
    model_path : Path
    all_params : dict
    metric_file_name : Path
    target_column : str
    mtflow_uri : str

In [17]:
from src.mlploject.constants import *
from src.mlploject.utils.common import read_yaml, save_json, create_directories

In [18]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.Elasticnet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params= params,
            metric_file_name= config.metric_file_name,
            target_column= schema.name,
            mtflow_uri= "https://dagshub.com/nishan129/machine-learning-mlflow.mlflow"
        )
        
        return model_evaluation_config

In [19]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [20]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mse = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        
        return rmse, mse, r2
    
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]
        
        mlflow.set_registry_uri(self.config.mtflow_uri)
        tracking_url_type_stor = urlparse(mlflow.get_tracking_uri()).scheme
        
        
        with mlflow.start_run():
            
            predict_qualities = model.predict(test_x)
            
            (rmse, mse, r2) = self.eval_metrics(test_y, predict_qualities)
            
            # saving matrics as loacal
            
            score = {"rmse": rmse, "mse": mse, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=score)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2",r2)
            mlflow.log_metric("mae", mse)
            
            
            # model registry does not work with file store
            
            if tracking_url_type_stor != "file":
                
                
                
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElsticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
            

In [21]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
    
except Exception as e:
    raise e
    

d:\MLproject\First_project\Project\machine-learning-mlflow\mlop\lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
d:\MLproject\First_project\Project\machine-learning-mlflow\mlop\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'ElsticnetModel' already exists. Creating a new version of this model...
2024/03/06 14:15:23 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElsticnetModel, v